# Verified Arithmetic in Transformers - Describe Model Algorithm

This Colab describes the algorithm of Transformer models in terms of model behaviours and algorithmic sub-tasks (analysed in other Colabs and stored in JSON files).

The models perform integer addition and/or subtraction e.g. 133357+182243=+0315600 and 123450-345670=-0123230. Each digit is a separate token. For 6 digit questions, the model is given 14 "question" (input) tokens, and must then predict the corresponding 8 "answer" (output) tokens.

This Colab follows on from:
- https://github.com/PhilipQuirke/verified_transformers/blob/main/notebooks/VerifiedArithmeticTrain.ipynb trained the models. It outputs model_name.pth and model_name_train.json
- https://github.com/PhilipQuirke/verified_transformers/blob/main/notebooks/VerifiedArithmeticAnalyse.ipynb analyzes the models. It documents their sub-tasks in model_name_behavior.json model_name_maths.json

This Colab loads the above json files from HuggingFace repository https://huggingface.co/PhilipQuirke/VerifiedArithmetic/raw/main


## Tips for using the Colab
 * You can run and alter the code in this CoLab notebook yourself in Google CoLab ( https://colab.research.google.com/ ).
 * To run the notebook, in Google CoLab, **you will need to** go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.
 * Some graphs are interactive!
 * Use the table of contents pane in the sidebar to navigate.
 * Collapse irrelevant sections with the dropdown arrows.
 * Search the page using the search in the sidebar, not CTRL+F.

# Part 0: Import libraries
Imports standard libraries.

Imports "verified_transformer" public library as "qt". This library is specific to this CoLab's "QuantaTool" approach to transformer analysis. Refer to [README.md](https://github.com/PhilipQuirke/verified_transformers/blob/main/README.md) for more detail.

In [ ]:
DEVELOPMENT_MODE = True
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    !pip install matplotlib

    !pip install kaleido
    !pip install transformer_lens
    !pip install torchtyping

    !pip install numpy
    #!pip install scikit-learn
    !pip install huggingface_hub

except:
    IN_COLAB = False

    def setup_jupyter(install_libraries=False):
        if install_libraries:
            !pip install matplotlib==3.8.4
            !pip install kaleido==0.2.1
            !pip install transformer_lens==1.15.0
            !pip install torchtyping==0.1.4
            !pip install numpy==1.26.4
            !pip install plotly==5.20.0
            !pip install pytest==8.1.1
            #!pip install scikit-learn==1.4.1.post1

        print("Running as a Jupyter notebook - intended for development only!")
        from IPython import get_ipython

        ipython = get_ipython()
        # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
        ipython.magic("load_ext autoreload")
        ipython.magic("autoreload 2")

    # setup_jupyter(install_libraries=True)   # Uncomment if you need to install libraries in notebook.
    setup_jupyter(install_libraries=False)

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import kaleido
import plotly.io as pio

if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

In [ ]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [ ]:
import json
import torch
import torch.nn.functional as F
import numpy as np
import random
import itertools
import re
from enum import Enum

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import textwrap
from huggingface_hub import hf_hub_download

In [ ]:
!pip install scikit-optimize

import re
import sklearn # Aka scikit.learn
import skopt # Aka scikit.optimize

In [ ]:
# Refer https://github.com/PhilipQuirke/verified_transformers/blob/main/README.md
!pip install --upgrade git+https://github.com/PhilipQuirke/verified_transformers.git  # Specify @branch if testing a specific branch
import QuantaTools as qt

# Part 1A: Configuration

Which existing model do we want to analyze?

The existing model weightings created by the sister Colab [VerifiedArithmeticTrain](https://github.com/PhilipQuirke/transformer-maths/blob/main/assets/VerifiedArithmeticTrain.ipynb) are loaded from HuggingFace (in Part 5). Refer https://github.com/PhilipQuirke/verified_transformers/blob/main/README.md for more detail.

In [ ]:
# Singleton QuantaTool "main" configuration class. MathsConfig is derived from the chain AlgoConfig > UsefulConfig > ModelConfig
cfg = qt.MathsConfig()

# Which model do we want to analyze? Uncomment one line:

# Addition models
#cfg.set_model_names( "add_d5_l1_h3_t15K_s372001" )  # AddAccuracy=Two9s. Inaccurate as only has one layer. Can predict S0, S1 and S2 complexity questions.
#cfg.set_model_names( "add_d5_l2_h3_t15K_s372001" )  # AddAccuracy=Six9s. AvgFinalLoss=1.6e-08
#cfg.set_model_names( "add_d5_l2_h3_t40K_s372001" )  # AddAccuracy=Six9s. AvgFinalLoss=2e-09. Fewest nodes
#cfg.set_model_names( "add_d6_l2_h3_t15K_s372001" )  # AddAccuracy=Six9s. AvgFinalLoss=1.7e-08. MAIN FOCUS
#cfg.set_model_names( "add_d6_l2_h3_t20K_s173289" )  # AddAccuracy=Six9s. AvgFinalLoss=1.5e-08
#cfg.set_model_names( "add_d6_l2_h3_t20K_s572091" )  # AddAccuracy=Six9s. AvgFinalLoss=7e-09
#cfg.set_model_names( "add_d6_l2_h3_t40K_s372001" )  # AddAccuracy=Six9s. AvgFinalLoss 2e-09. Fewest nodes
#cfg.set_model_names( "add_d10_l2_h3_t40K_s572091" ) # AddAccuracy=Six9s. AvgFinalLoss=7e-09. (1/M fail: 0000000555+0000000445=+00000001000 ModelAnswer: +00000000900)
#cfg.set_model_names( "add_d10_l2_h3_t40K_gf_s572091" ) # AddAccuracy=Six9s. AvgFinalLoss=3.5-09. GrokFast.

# Subtraction model
#cfg.set_model_names( "sub_d6_l2_h3_t30K_s372001" )  # SubAccuracy=Six9s. AvgFinalLoss=5.8e-06
#cfg.set_model_names( "sub_d10_l2_h3_t75K_s173289" )  # SubAccuracy=Two9s. (6672/M fails) AvgFinalLoss=0.002002022
#cfg.set_model_names( "sub_d10_l2_h3_t75K_gf_s173289" )  # SubAccuracy=Two9s. GrokFast. (5246/M fails) AvgFinalLoss=0.001197

# Mixed (addition and subtraction) model
#cfg.set_model_names( "mix_d6_l3_h4_t40K_s372001" )  # Add/SubAccuracy=Six9s/Six9s. AvgFinalLoss=5e-09. (1/M fail: 463687+166096=+0629783 ModelAnswer: +0639783)
#cfg.set_model_names( "mix_d10_l3_h4_t75K_s173289" )  # Add/SubAccuracy=Five9s/Two9s. AvgFinalLoss=1.125e-06 (2/M fail: 3301956441+6198944455=+09500900896 ModelAnswer: +09500800896) (295/M fail: 8531063649-0531031548=+08000032101 ModelAnswer: +07900032101)

# Mixed models initialized with addition model
#cfg.set_model_names( "ins1_mix_d6_l2_h3_t40K_s572091" )  # Add/SubAccuracy=Six9s/Five9s. AvgLoss = 2.4e-08 (5/M fails e.g. 565000-364538=+0200462 ModelAnswer: +0100462)
#cfg.set_model_names( "ins1_mix_d6_l3_h3_t40K_s572091" )  # Add/SubAccuracy=Six9s/Five9s. AvgFinalLoss=1.8e-08. (3/M fails e.g. 072074-272074=-0200000 ModelAnswer: +0200000)
#cfg.set_model_names( "ins1_mix_d6_l3_h3_t80K_s572091" )  # Add/SubAccuracy=Six9s/Five9s AvgLoss = 1.6e-08 (3/M fails e.g. 229672-229678=-0000006 ModelAnswer: +0000006)
cfg.set_model_names( "ins1_mix_d6_l3_h4_t40K_s372001" )  # Add/SubAccuracy=Six9s/Six9s. AvgFinalLoss=8e-09. MAIN FOCUS
#cfg.set_model_names( "ins1_mix_d6_l3_h4_t40K_s173289" )  # Add/SubAccuracy=Five9s/Five9s. AvgFinalLoss=1.4e-08. (3/M fails e.g. 850038+159060=+1009098 ModelAnswer: +0009098) (2 fails e.g. 77285-477285=+0100000 Q: ModelAnswer: +0000000).
#cfg.set_model_names( "ins1_mix_d6_l3_h4_t50K_s572091" )  # Add/SubAccuracy=Six9s/Five9s. AvgFinalLoss=2.9e-08. (4/M fails e.g. 986887-286887=+0700000 ModelAnswer: +0600000)
#cfg.set_model_names( "ins1_mix_d10_l3_h3_t50K_s572091" )  # Add/SubAccuracy=Five9s/Five9s. AvgFinalLoss 6.3e-07  (6/M fails e.g. 5068283822+4931712829=+09999996651 ModelAnswer: +19099996651) (7/M fails e.g. 3761900218-0761808615=+03000091603 ModelAnswer: +02000091603)
#cfg.set_model_names( "ins1_mix_d10_l3_h3_t50K_gf_s572091" ) # Add/SubAccuracy=???/???. GrokFast. Unenriched data.

# Mixed model initialized with addition model. Reset useful heads every 100 epochs.
#cfg.set_model_names( "ins2_mix_d6_l4_h4_t40K_s372001" )  # Add/SubAccuracy=Five9s/Five9s. AvgFinalLoss=1.7e-08. (3/M fails e.g. 530757+460849=+0991606 ModelAnswer: +0091606) (8 fails e.g. 261926-161857=+0100069 ModelAnswer: +0000069)

# Mixed model initialized with addition model. Reset useful heads & MLPs every 100 epochs.
#cfg.set_model_names( "ins3_mix_d6_l4_h3_t40K_s372001" )  # Add/SubAccuracy=Four9s/Two9s. AvgFinalLoss=3.0e-04. (17/M fails e.g. 273257+056745=+0330002 ModelAnswer: +0320002) (3120 fails e,g. 09075-212133=-0003058 ModelAnswer: +0003058)

# Mixed models initialized with addition model.
#cfg.set_model_names( "ins4_mix_d6_l3_h4_t30K_s775824" )  # Add/SubAccuracy=???/??? TODO
#cfg.set_model_names( "ins4_mix_d6_l2_h4_t30K_s775824" )  # Add/SubAccuracy=???/??? TODO

# Part 1B: Configuration: Input and Output file names



In [ ]:
cfg.batch_size = 512 # Default analysis batch size
if cfg.n_layers >= 3 and cfg.n_heads >= 4:
  cfg.batch_size = 256 # Reduce batch size to avoid memory constraint issues.

In [ ]:
main_fname_pth = cfg.model_name + '.pth'
main_fname_train_json = cfg.model_name + '_train.json'
main_fname_behavior_json = cfg.model_name + '_behavior.json'
main_fname_maths_json = cfg.model_name + '_maths.json'
main_repo_name="PhilipQuirke/VerifiedArithmetic"

In [ ]:
# Update "cfg" with additional training (including cfg.insert_*) config information from stored file:
#      https://huggingface.co/PhilipQuirke/VerifiedArithmetic/raw/main/ins1_mix_d6_l3_h4_t40K_s372001_train.json"
training_data_json = qt.download_huggingface_json(main_repo_name, main_fname_train_json)
training_loss_list = qt.load_training_json(cfg, training_data_json)
print('Loaded main model training config / loss from', main_repo_name, main_fname_train_json)

In [ ]:
def print_config():
  print("%Add=", cfg.perc_add, "%Sub=", cfg.perc_sub, "%Mult=", cfg.perc_mult, "InsertMode=", cfg.insert_mode, "File=", cfg.model_name)

In [ ]:
print_config()
print("weight_decay=", cfg.weight_decay, "lr=", cfg.lr, "batch_size=", cfg.batch_size)
print('Main model will be read from HuggingLab file', main_repo_name, main_fname_pth)
print('Main model training config / loss will be read from HuggingLab file', main_fname_train_json)
print('Main model behavior analysis tags will be read from HuggingLab file', main_fname_behavior_json)
print('Main model maths analysis tags will be read from HuggingLab file', main_fname_maths_json)

In [ ]:
# Singleton QuantaTool "ablation intervention" configuration class
acfg = qt.acfg

# Part 3A: Set Up: Vocabulary / Embedding / Unembedding

  

In [ ]:
qt.set_maths_vocabulary(cfg)
qt.set_maths_question_meanings(cfg)
print(cfg.token_position_meanings)

# Part 4: Results: Model training loss graphs

In [ ]:
print_config()
print( "Avg loss over last 5 epochs", cfg.avg_final_loss)
print( "Final epoch loss", cfg.final_loss)

In [ ]:
if training_loss_list:
    answer_digits = cfg.n_digits + 1
    title_font_size=32
    tick_font_size=24

    qt.plot_loss_lines(cfg, 1500, [training_loss_list[:1500]], labels = ['All'], log_y=False,
                       title='Training Loss', title_font_size=title_font_size, tick_font_size=tick_font_size)

    full_title, fig = qt.plot_loss_lines(cfg, cfg.n_training_steps, [training_loss_list], labels = ['All'], log_y=True,
                                         title='Training Loss', title_font_size=title_font_size, tick_font_size=tick_font_size)
    pio.write_image(fig, cfg.model_name + '_LogTrainingLoss.' + cfg.graph_file_suffix)

# Part 6A: Set Up: Load nodes and behaviour tags
Load the useful nodes and associated behaviour tags from a JSON file stored on HuggingFace

In [ ]:
print("Loading useful node list with behavior tags from HuggingFace", main_repo_name, main_fname_behavior_json)

file_path = hf_hub_download(repo_id=main_repo_name, filename=main_fname_behavior_json, revision="main")

cfg.useful_nodes.load_nodes(file_path)

# Part 6B: Results: Show nodes and behaviour tags

In [ ]:
cfg.useful_nodes.sort_nodes()
cfg.useful_nodes.print_node_tags()

In [ ]:
def show_quanta_map( title, major_tag : qt.QType, minor_tag : str, get_node_details,  \
        image_width_inches : int = 9, image_height_inches : int = 6,
        blue_shades : bool = True, cell_num_shades : int = 6,
        filters : qt.FilterNode = None, cell_fontsize : int = 9,
        combine_identical_cells : bool = True, show_perc_circles : bool = False ): \

  test_nodes = cfg.useful_nodes
  if filters is not None:
    test_nodes = qt.filter_nodes(test_nodes, filters)

  ax1, quanta_results, num_results = qt.calc_quanta_map(
      cfg, blue_shades, cell_num_shades,
      test_nodes, major_tag.value, minor_tag, get_node_details,
      cell_fontsize, combine_identical_cells, show_perc_circles,
      image_width_inches, image_height_inches )

  if num_results > 0:
    plt.show()

# Part 16A: Results: Show failure percentage map

Show the percentage failure rate (incorrect prediction) when individual Attention Heads and MLPs are ablated.

In [ ]:
show_quanta_map( "Failure Frequency Behavior Per Node",
                qt.QType.FAIL, "", qt.get_quanta_fail_perc,
                image_height_inches = 2 * cfg.n_layers,
                cell_num_shades = qt.FAIL_SHADES, combine_identical_cells = False, show_perc_circles = True)

# Part 16B - Show answer impact behavior map

Show the purpose of each useful cell by impact on the answer digits A0 to Amax.


In [ ]:
show_quanta_map( "Answer Impact Behavior Per Node",
                qt.QType.IMPACT, "", qt.get_quanta_impact,
                image_height_inches = 2 * cfg.n_layers,
                cell_num_shades = cfg.num_answer_positions)

# Part 16C: Result: Show attention map

Show attention quanta of useful heads

In [ ]:
# Only maps attention heads, not MLP layers
show_quanta_map( "Attention Behavior Per Head",
                qt.QType.ATTN, "", qt.get_quanta_attention,
                image_height_inches = 3 * cfg.n_layers,
                cell_num_shades = qt.ATTN_SHADES )

# Part 16C - Show question complexity map


In [ ]:
if cfg.perc_sub > 0:
  # For each useful cell, show if addition (S), positive-answer subtraction (M) and negative-answer subtraction (N) questions relies on the node.
  show_quanta_map( "Maths Operation Coverage",
                  qt.QType.MATH, "", qt.get_maths_operation_complexity,
                  image_height_inches = 1.75 * cfg.n_layers,
                  blue_shades = False, cell_num_shades = 4, combine_identical_cells = False)

In [ ]:
if cfg.perc_add > 0:
  # For each useful cell, show the minimum addition question complexity that relies on the node, as measured using quanta S0, S1, S2, ...
  show_quanta_map( "Addition Min-Complexity",
                  qt.QType.MATH_ADD, qt.MathsBehavior.ADD_COMPLEXITY_PREFIX.value, qt.get_maths_min_complexity,
                  image_height_inches = 1.25 * cfg.n_layers,
                  blue_shades = False, cell_num_shades = qt.MATH_ADD_SHADES)

In [ ]:
if cfg.perc_sub > 0:
  # For each useful cell, show the minimum "positive-answer subtraction" question complexity that relies on the node, as measured using quanta M0, M1, M2, ...
  show_quanta_map( "Positive-answer Subtraction Min-Complexity",
                  qt.QType.MATH_SUB, qt.MathsBehavior.SUB_COMPLEXITY_PREFIX.value, qt.get_maths_min_complexity,
                  image_height_inches = 1.5 * cfg.n_layers,
                  blue_shades = False, cell_num_shades = qt.MATH_SUB_SHADES)

In [ ]:
if cfg.perc_sub > 0:
  # For each useful cell, show the minimum "negative-answer subtraction" question complexity that relies on the node, as measured using quanta N0, N1, N2, ...
  show_quanta_map( "Negative-answer Subtraction Min-Complexity",
                  qt.QType.MATH_NEG, qt.MathsBehavior.NEG_COMPLEXITY_PREFIX.value, qt.get_maths_min_complexity,
                  image_height_inches = 1.5 * cfg.n_layers,
                  blue_shades = False, cell_num_shades = qt.MATH_SUB_SHADES)

# Part 17: Set Up: Load maths sub-task tags from json file

Load the useful nodes maths sub-task tags from a JSON file stored on HuggingFace

In [ ]:
main_repo_name="PhilipQuirke/VerifiedArithmetic"
print("Loading maths sub-tasks from HuggingFace", main_repo_name, main_fname_maths_json)

file_path = hf_hub_download(repo_id=main_repo_name, filename=main_fname_maths_json, revision="main")

cfg.useful_nodes.load_nodes(file_path)

# Part 23A: Show algorithm quanta map

Plot the "maths" tags as a quanta map. This is an automatically generated partial explanation of the model algorithm.

Nodes with multiple tags perform multiple tasks.

In [ ]:
qt.print_algo_purpose_results(cfg)

In [ ]:
show_quanta_map( "Maths Purpose Per Node", qt.QType.ALGO, "", qt.get_quanta_binary,
                #image_width_inches = 11,
                cell_num_shades = 2)

# Part 23B: Show known quanta per answer digit

Each of the late positions are soley focused on calculating one answer digit. Show the data have we collected on late answer digit.  



In [ ]:
for position in range(cfg.num_question_positions + 1, cfg.n_ctx - 1):
  print("Position:", position)

  # Calculate a table of the known quanta for the specified position for each late token position
  qt.calc_maths_quanta_for_position_nodes(cfg, position)

  plt.show()

# Part 24: Show maths tags

Show a list of the nodes that have proved useful in calculations, together with data on the nodes behavior and algorithmic purposes.


In [ ]:
cfg.useful_nodes.print_node_tags(qt.QType.ALGO.value, "", False)

In [ ]:
mixed_model = cfg.model_name.startswith("ins1_mix_d6_l3_h4_t40K") or cfg.model_name.startswith("ins2_mix_d6_l4_h4_t40K") or cfg.model_name.startswith("ins3_mix_d6_l4_h3_t40K")

# Part 25 : Results: Test Algorithm - Addition

In [ ]:
print_config()

In [ ]:
# For answer digits (excluding Amax), a An.SA node is needed before the answer digit is revealed
def algo_task_search(algo_nodes, from_digit, to_digit, filter_function, mandatory : bool = True):
    impact_digit = from_digit
    while impact_digit <= to_digit:
        cfg.test_algo_clause(algo_nodes, filter_function(impact_digit), mandatory)
        impact_digit += 1

In [ ]:
# Read as: And( HasAlgoTag:A4.SA, Position<=A3 )
def algo_sa_filter(impact_digit):
    return qt.FilterAnd(qt.FilterAlgo(qt.add_sa_functions.tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX))

# For each question digit, an An.SA node exists before the answer digit is revealed
def algo_sa_search(algo_nodes):
    algo_task_search(algo_nodes, 0, cfg.n_digits-1, algo_sa_filter)


# Read as: And( HasAlgoTag:A4.SC, Position<=A3 )
def algo_sc_filter(impact_digit):
    return qt.FilterAnd(qt.FilterAlgo(qt.add_sc_functions.tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX))

# For each question digit, except A0, an An.SC node exists before the answer digit is revealed
def algo_sc_search(algo_nodes, mandatory : bool = True):
    algo_task_search(algo_nodes, 1, cfg.n_digits-1, algo_sc_filter, mandatory)


# Read as: And( HasAlgoTag:A4.SS, Position<=A3 )
def algo_ss_filter(impact_digit):
    return qt.FilterAnd(qt.FilterAlgo(qt.add_ss_functions.tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX))

# For each question digit, except A0 and A1, an An.SS node exists before the answer digit is revealed
def algo_ss_search(algo_nodes):
    algo_task_search(algo_nodes, 2, cfg.n_digits-1, algo_ss_filter)


# Read as: And( HasAlgoTag:A4.ST, Position<=Amax )
def algo_st_filter(impact_digit):
    return qt.FilterAnd(qt.FilterAlgo(qt.add_st_functions.tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(cfg.n_digits+1), qt.QCondition.MAX))

# For each question digit, except A0, an An.ST node exists by Amax
def algo_st_search(algo_nodes):
    algo_task_search(algo_nodes, 2, cfg.n_digits-1, algo_st_filter)


# The nodes that implement the ST sub-task must be sequenced so that the SV values can be calculated.
def algo_sv_search(algo_nodes):
    # TODO: It is not clear what this "sequencing" condition is
    pass

## Part25A : 1-layer addition models use sub-tasks An.SA, An.SC, An.SS

A 1-layer model can do 99% of addition questions using SA, SC and SS sub-tasks (as per Paper 1).

For add_d5_l1_h3_t15K_s372001 below search says: 14/15 heads have purpose assigned. 0/6 neurons have purpose assigned.

In [ ]:
if cfg.perc_add > 0 and cfg.n_layers == 1 :

    algo_nodes = cfg.start_algorithm_test(acfg)

    algo_sa_search(algo_nodes)
    algo_sc_search(algo_nodes, mandatory=True)
    algo_ss_search(algo_nodes)
    algo_sv_search(algo_nodes)

    cfg.print_algo_clause_results()
    cfg.print_algo_purpose_results(algo_nodes)

## Part25B : 2-layer addition models use sub-tasks An.SA, An.ST and maybe An.SC

A 2-layer model can do 99.9999% of addition questions using SA and ST sub-tasks (as per Paper 2). It may use (redundant) SC sub-tasks.

For add_d6_l2_h3_t15K below search says: 21/31 heads have purpose assigned. 0/17 neurons have purpose assigned.

TODO: 4 other models.

In [ ]:
if cfg.perc_add > 0 and cfg.n_layers >= 2 :

  algo_nodes = cfg.start_algorithm_test(acfg)

  algo_sa_search(algo_nodes)
  algo_st_search(algo_nodes)
  # SC does the same job as but is less accurate than ST. So the SC nodes are redundant and may be optimised out.
  algo_sc_search(algo_nodes, mandatory=False)

  cfg.print_algo_clause_results()
  cfg.print_algo_purpose_results(algo_nodes)

# Part 26: Results: Test Algorithm - Subtraction

## Part 26A : 1-layer subtraction uses sub-tasks MD, MB & MZ

A 1-layer model can do 99% of subtraction questions using MD, MB and MZ sub-tasks (as per Paper 1).

In [ ]:
# Read as: And( HasAlgoTag:A4.MD, Position<=A3 )
def algo_md_filter(impact_digit):
    return qt.FilterAnd(qt.FilterAlgo(qt.sub_md_functions.tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX))

# For each question digit, an An.MD node exists before the answer digit is revealed
def algo_md_search(algo_nodes):
    algo_task_search(algo_nodes, 0, cfg.n_digits-1, algo_md_filter)


# Read as: And( HasAlgoTag:A4.MB, Position<=A3 )
def algo_mb_filter(impact_digit):
    return qt.FilterAnd(qt.FilterAlgo(qt.sub_mb_functions.tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX))

# For each question digit, an An.MB node exists before the answer digit is revealed
def algo_mb_search(algo_nodes, mandatory : bool = True):
    algo_task_search(algo_nodes, 0, cfg.n_digits-1, algo_mb_filter, mandatory)


In [ ]:
if cfg.perc_sub > 0 and cfg.n_layers == 1 :

    algo_nodes = cfg.start_algorithm_test(acfg)

    algo_md_search(algo_nodes)
    algo_mb_search(algo_nodes, mandatory = True)

    cfg.print_algo_clause_results()
    cfg.print_algo_purpose_results(algo_nodes)

## Part 26B: Test Algorithm - Subtraction

To accurately predict if the answer sign is + or - the model must calculate if D < D'. To calculate this, the model must calculate Dn < D'n or (Dn = D'n and (Dn-1 < D'n-1 or (Dn-2 = D'n-1 and ( etc. It must predict this before the answer sign is revealed.

We expect to see nodes useful in negative answer questions, with PCA bigram (or trigram) outputs, attending to these input pairs, evaluated in this order, before the answer sign is revealed.

In [ ]:
def algo_mt_filter(impact_digit,sign_position):
    return qt.FilterAnd(
      qt.FilterAlgo(qt.sub_mt_functions.tag(impact_digit)),
      qt.FilterPosition(sign_position, qt.QCondition.MAX))


# For answer digits (excluding Amax), An.MT is needed before the answer digit is revealed
def algo_mt_search(algo_nodes):
  mt_locations = {}

  sign_position = cfg.an_to_position_name(cfg.n_digits+1)

  for impact_digit in range(cfg.n_digits):
    # For answer digits (excluding the +/- answer sign and 0 or 1 first answer digit), An.SC is calculated before the answer sign is revealed
    position = cfg.test_algo_clause(algo_nodes, algo_mt_filter(impact_digit, sign_position))
    mt_locations[impact_digit] = position

  # Check that mt_locations[6] < mt_locations[5] < mt_locations[4] < etc
  print("MT Locations:", mt_locations)
  for impact_digit in range(cfg.n_digits):
    if impact_digit > 0:
      sc1 = mt_locations[impact_digit]
      sc2 = mt_locations[impact_digit-1]
      description = f"MT Ordering: A{impact_digit}={sc1}, A{impact_digit-1}={sc2}"
      cfg.test_algo_logic(description, sc1 >= 0 and sc2 >= 0 and sc1 < sc2)

In [ ]:
if cfg.perc_sub > 0 and cfg.n_layers >= 2 :

    algo_nodes = cfg.start_algorithm_test(acfg)

    algo_md_search(algo_nodes)
    algo_mt_search(algo_nodes)
    algo_mb_search(algo_nodes, mandatory = False) # TODO: Is this true?

    cfg.print_algo_clause_results()
    cfg.print_algo_purpose_results(algo_nodes)

# Part 27: Test Algorithm - Mixed Addition and Subtraction model

What algorithm do mixed models use to perform both addition and subtraction? Our working hypothesis is Hypothesis 2 as described in https://github.com/PhilipQuirke/verified_transformers/blob/main/mixed_model.md





## Part 27A: Calculating answer digit A2 in token position A3

The below graph displays the same (behavior and algorithm) data as the quanta maps. Refer https://github.com/PhilipQuirke/verified_transformers/blob/main/mixed_model.md section 27A for more explanation.

    


In [ ]:
qt.calc_maths_quanta_for_position_nodes(cfg, 18)
plt.show()

## Part 27.H3 Calculate whether D > D' (using NG tasks)

In [ ]:
filters = qt.FilterContains(qt.QType.MATH_NEG, "")

#print("NG tagged nodes:", qt.filter_nodes( cfg.useful_nodes, filters ).get_node_names())

show_quanta_map( "Subtraction Behavior NG Nodes", qt.QType.MATH_SUB, "", qt.get_maths_min_complexity, 9, 6, filters=filters, blue_shades=False, cell_num_shades=2)
show_quanta_map( "Attention Behavior Per NG Head", qt.QType.ATTN, "", qt.get_quanta_attention, 9, 8, filters=filters, cell_num_shades=10)
show_quanta_map( "Maths Purpose Per NG Node", qt.QType.ALGO, "", qt.get_quanta_binary, 9, 6, filters=filters, cell_num_shades=2)

In [ ]:
if mixed_model:
  algo_nodes = cfg.start_algorithm_test(acfg)

  algo_sc_search(algo_nodes)
  cfg.print_algo_clause_results()

  cfg.print_algo_purpose_results(algo_nodes)

# Part 30: Unit Test automated searches

In [ ]:
def check_algo_tag_exists(node_location_as_str, the_tags ):
  node_location = qt.str_to_node_location(node_location_as_str)
  node = cfg.useful_nodes.get_node(node_location)
  if node is None:
      print( "Node", node_location_as_str, "is missing")
  else:
      for the_tag in the_tags:
          if not node.contains_tag(qt.QType.ALGO.value, the_tag):
              print( "Node", node.name(), "is missing tag", the_tag, "It has tags:", node.tags )

In [ ]:
print(cfg.model_name)

if cfg.model_name.startswith('add_d6_l2_h3_t15K'):
  check_algo_tag_exists('P11L0H0', ['A2.ST'] )
  check_algo_tag_exists('P12L0H0', ['A3.ST'] )
  check_algo_tag_exists('P14L0H0', ['A5.SS', 'A4.ST'] )
  check_algo_tag_exists('P14L0H2', ['A5.SC', 'A5.ST'] )
  check_algo_tag_exists('P14L1H1', ['OPR'] )
  check_algo_tag_exists('P15L0H0', ['A4.SC'] )
  check_algo_tag_exists('P15L0H1', ['A5.SA'] )
  check_algo_tag_exists('P15L0H2', ['A5.SA'] )
  check_algo_tag_exists('P16L0H0', ['A3.SC'] )
  check_algo_tag_exists('P16L0H1', ['A4.SA'] )
  check_algo_tag_exists('P16L0H2', ['A4.SA'] )
  check_algo_tag_exists('P17L0H0', ['A2.SC'] )
  check_algo_tag_exists('P17L0H1', ['A3.SA'] )
  check_algo_tag_exists('P17L0H2', ['A3.SA'] )
  check_algo_tag_exists('P18L0H0', ['A1.SC'] )
  check_algo_tag_exists('P18L0H1', ['A2.SA'] )
  check_algo_tag_exists('P18L0H2', ['A2.SA'] )
  check_algo_tag_exists('P19L0H0', ['A0.SC'] )
  check_algo_tag_exists('P19L0H1', ['A1.SA'] )
  check_algo_tag_exists('P19L0H2', ['A1.SA'] )
  check_algo_tag_exists('P20L0H1', ['A0.SA'] )
  check_algo_tag_exists('P20L0H2', ['A0.SA'] )

if cfg.model_name.startswith('mix_d6_l3_h4_t40K'):
  check_algo_tag_exists('P8L0H1', ['OPR'] )
  check_algo_tag_exists('P13L2H0', ['A7.NG'] )
  check_algo_tag_exists('P15L0H0', ['A5.SA', 'A5.MD'] )
  check_algo_tag_exists('P15L0H3', ['A5.SA', 'A5.MD'] )
  check_algo_tag_exists('P16L0H3', ['A4.SA.A4', 'A4.MD.A4'] )
  check_algo_tag_exists('P17L0H1', ['A3.NG'] )
  check_algo_tag_exists('P17L0H3', ['A3.SA.A3', 'A3.MD.A3'] )
  check_algo_tag_exists('P18L0H1', ['A2.NG'] )
  check_algo_tag_exists('P18L0H3', ['A2.SA.A2', 'A2.MD.A2'] )
  check_algo_tag_exists('P19L0H1', ['A1.NG'] )
  check_algo_tag_exists('P19L0H3', ['A1.SA.A1', 'A1.MD.A1'] )
  check_algo_tag_exists('P20L0H0', ['A0.SA', 'A0.MD'] )
  check_algo_tag_exists('P20L0H3', ['A0.SA', 'A0.MD'] )
  check_algo_tag_exists('P20L2H1', ['A0.NG'] )

if cfg.model_name.startswith('ins1_mix_d6_l3_h4_t40K'):
  check_algo_tag_exists('P6L0H0', ['OPR'] )
  check_algo_tag_exists('P9L0H0', ['A4.MT'] )
  check_algo_tag_exists('P9L0H1', ['A4.ST'] )
  check_algo_tag_exists('P10L0H1', ['A2.MT'] )
  check_algo_tag_exists('P10L0H3', ['OPR'] )
  check_algo_tag_exists('P12L0H0', ['A3.MT'] )
  check_algo_tag_exists('P12L0H1', ['A3.ST'] )
  check_algo_tag_exists('P12L0H3', ['OPR'] )
  check_algo_tag_exists('P12L1H2', ['A1.MT'] )
  check_algo_tag_exists('P13L0H3', ['OPR'] )
  check_algo_tag_exists('P13L1H0', ['OPR'] )
  check_algo_tag_exists('P13L2H0', ['OPR'] )
  check_algo_tag_exists('P14L0H0', ['A5.SS', 'OPR'] )
  check_algo_tag_exists('P14L0H1', ['OPR'] )
  check_algo_tag_exists('P14L0H2', ['A5.SC', 'A5.ST', 'SGN'] )
  check_algo_tag_exists('P14L1H2', ['SGN'] )
  check_algo_tag_exists('P14L1H3', ['SGN'] )
  check_algo_tag_exists('P15L0H0', ['A4.SC'] )
  check_algo_tag_exists('P15L0H1', ['A5.SA', 'A5.MD'] )
  check_algo_tag_exists('P15L0H2', ['A5.SA', 'A5.MD'] )
  check_algo_tag_exists('P15L0H3', ['SGN'] )
  check_algo_tag_exists('P16L0H0', ['A3.SC'] )
  check_algo_tag_exists('P16L0H1', ['A4.SA', 'A4.MD', 'A4.ND'] )
  check_algo_tag_exists('P16L0H2', ['A4.SA', 'A4.MD', 'A4.ND'] )
  check_algo_tag_exists('P16L0H3', ['SGN'] )
  check_algo_tag_exists('P16L1H0', ['SGN'] )
  check_algo_tag_exists('P16L2H0', ['SGN'] )
  check_algo_tag_exists('P17L0H0', ['A2.SC'] )
  check_algo_tag_exists('P17L0H1', ['A3.SA', 'A3.MD', 'A3.ND'] )
  check_algo_tag_exists('P17L0H2', ['A3.SA', 'A3.MD', 'A3.ND'] )
  check_algo_tag_exists('P17L0H3', ['SGN'] )
  check_algo_tag_exists('P18L0H0', ['A1.SC'] )
  check_algo_tag_exists('P18L0H1', ['A2.SA', 'A2.MD', 'A2.ND'] )
  check_algo_tag_exists('P18L0H2', ['A2.SA', 'A2.MD', 'A2.ND'] )
  check_algo_tag_exists('P19L0H0', ['A0.MB.A1'] )
  check_algo_tag_exists('P19L0H1', ['A1.SA', 'A1.MD', 'A1.ND'] )
  check_algo_tag_exists('P19L0H2', ['A1.SA', 'A1.MD', 'A1.ND'] )
  check_algo_tag_exists('P20L0H1', ['A0.SA', 'A0.MD', 'A0.ND'] )
  check_algo_tag_exists('P20L0H2', ['A0.SA', 'A0.MD', 'A0.ND'] )